# INF170 - Mandatory Assignment: Supply Chain Network Design
> **Name:** Trym Døssland Bjerkvik <br>
> **Date:** 03.11.2023

An optimization and modelling project for designing a cost-efficient bioethanol supply chain network in Texas as part of the INF170 course.

## Table of Contents
    
   * [0. Imports and Data Loading](#0.-Imports-and-Data-Loading)
   * [1. Task 1](#-1.-Task-1)
   * [2. Taks 2](#-2.-Task-2)
   * [3. Taks 3](#-3.-Task-3)

## 0. Imports and Data Loading

In [1]:
# Imports.
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import pandas as pd

In [2]:
# Load the data.
suppliers = pd.read_csv("suppliers.csv")
plants = pd.read_csv("plants.csv")
hubs = pd.read_csv("hubs.csv")
roads_s_p = pd.read_csv("roads_s_p.csv")
roads_s_h = pd.read_csv("roads_s_h.csv")
railroads_h_p = pd.read_csv("railroads_h_p.csv")

In [3]:
print(suppliers.columns)
suppliers.head()

Index(['supplier', 'supply'], dtype='object')


,supplier,supply
0,48001,13131.971710
1,48003,1177.351950
2,48005,3854.618542
3,48007,308.182629
4,48009,19802.136510


In [4]:
print(plants.columns)
plants.head()

Index(['plant', 'plant_cost', 'plant_cap', 'yield_per_unit'], dtype='object')


,plant,plant_cost,plant_cap,yield_per_unit
0,541,130956797,152063705,232
1,542,130956797,152063705,232
2,543,130956797,152063705,232
3,544,130956797,152063705,232
4,545,130956797,152063705,232


In [5]:
print(hubs.columns)
hubs.head()

Index(['hub', 'hub_cost', 'hub_cap'], dtype='object')


,hub,hub_cost,hub_cap
0,17201,3476219,300000
1,17218,3476219,300000
2,17359,3476219,300000
3,17372,3476219,300000
4,17395,3476219,300000


In [6]:
print(roads_s_p.columns)
roads_s_p.head()

Index(['supplier', 'plant', 'dist_s_p', 'cost_per_unit_s_p', 'truck_cost_s_p',
       'truck_cap_s_p'],
      dtype='object')


,supplier,plant,dist_s_p,cost_per_unit_s_p,truck_cost_s_p,truck_cap_s_p
0,48001,541,397.819459,55.892588,10000,500
1,48001,542,636.639175,55.892588,10000,500
2,48001,543,288.171679,47.661277,10000,500
3,48001,544,550.495281,55.892588,10000,500
4,48001,545,636.639175,55.892588,10000,500


In [7]:
print(roads_s_h.columns)
roads_s_h.head()

Index(['supplier', 'hub', 'dist_s_h', 'cost_per_unit_s_h ', 'truck_cost_s_h',
       'truck_cap_s_h'],
      dtype='object')


,supplier,hub,dist_s_h,cost_per_unit_s_h,truck_cost_s_h,truck_cap_s_h
0,48001,17201,219.6090,40.580691,10000,500
1,48001,17218,264.7658,47.661277,10000,500
2,48001,17359,370.6750,64.267840,10000,500
3,48001,17372,420.0082,72.003286,10000,500
4,48001,17395,199.2915,37.394907,10000,500


In [8]:
# Removing trailing spaces from the column names.
roads_s_h.columns = [columns.strip() for columns in roads_s_h.columns]

In [9]:
print(railroads_h_p.columns)
railroads_h_p.head()

Index(['hub', 'plant', 'dist_h_p ', 'cost_per_unit_h_p ', 'train_cost_h_p ',
       'train_cap_h_p'],
      dtype='object')


,hub,plant,dist_h_p,cost_per_unit_h_p,train_cost_h_p,train_cap_h_p
0,17201,541,922.023251,17.095711,60000,20000
1,17201,542,1160.842967,20.923752,60000,20000
2,17201,543,186.924258,5.312809,60000,20000
3,17201,544,1074.699073,19.542951,60000,20000
4,17201,545,1160.842967,20.923752,60000,20000


In [10]:
# Removing trailing spaces from the column names.
railroads_h_p.columns = [columns.strip() for columns in railroads_h_p.columns]

## 1. Task 1

In [11]:
TOLERANCE = 1e-9  # Some values below this threshold will be considered as zero.

# Create a new model.
m1 = Model("DirectSupplyModel")

# Sets:
SSet = suppliers['supplier'].tolist() # Set of suppliers (counties).
PSet = plants['plant'].tolist() # Set of potential plants (biorefineries). 

# Parameters:
Supply = suppliers.set_index('supplier')['supply'].to_dict() # Supply of biomass from supplier s (tonne).
Yield = 232  # Conversion yield of bioethanol (liters per tonne of biomass).
PlantCap = 152063705 # Annual conversion capacity for each plant (liters).
Demand = 500000000  # Total bioethanol demand (production target) (liters).
InvCost = plants.set_index('plant')['plant_cost'].to_dict() # Cost to maintain a plant at location p.
TransCost = roads_s_p.set_index(['supplier', 'plant'])['cost_per_unit_s_p'].to_dict() # Transportation cost.
Dist = roads_s_p.set_index(['supplier', 'plant'])['dist_s_p'].to_dict() # Distance from s to p (km).
LoadCost = 10000  # Combined loading and unloading cost for each trip ($).
TruckCap = 500  # Capacity of truck (tonne).

# Decision variables:
x = m1.addVars(PSet, vtype=GRB.BINARY, name="x") # 1 if plant is built at location p, else 0.
y = m1.addVars(SSet, PSet, vtype=GRB.CONTINUOUS, name="y") # Amount of biomass transported from s to p. 
trips = m1.addVars(SSet, PSet, vtype=GRB.INTEGER, name="trips") # Number of truck trips needed.

# Objective function:
investment_cost = quicksum(InvCost[p] * x[p] for p in PSet) # Investment cost.
transport_cost = quicksum(TransCost[s,p] * Dist[s,p] * y[s,p] for s in SSet for p in PSet) # Transport cost.
loading_cost = quicksum(LoadCost * trips[s,p] for s in SSet for p in PSet) # Loading cost.
m1.setObjective(investment_cost + transport_cost + loading_cost, GRB.MINIMIZE)

# Constraints:
# Supply constraints: A supplier can only supply the amount of biomass that it has available.
for s in SSet:
    m1.addConstr(quicksum(y[s, p] for p in PSet) <= Supply[s], name=f"Supply_{s}")

# Plant constraints: A plant can only process biomass up to its capacity, and only after it is built. 
for p in PSet:
    m1.addConstr(quicksum(Yield * y[s,p] for s in SSet) <= PlantCap * x[p], name=f"Plant_{p}")

# Bioethanol production requirement: Ensure that the total bioethanol produced meets the demand.
m1.addConstr(quicksum(Yield * y[s, p] for s in SSet for p in PSet) >= Demand, name="Demand")
    
# Truck trips constraints: Ensures the number of truck trips needed.
for s in SSet:
    for p in PSet:
        m1.addConstr(y[s,p] <= TruckCap * trips[s,p], name=f"TruckTrips_{s}_{p}")

# Optimize the model.
m1.setParam('MIPGap', 1e-9)
m1.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-19
Set parameter MIPGap to value 1e-09
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 5 7600 6-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 42840 rows, 85003 columns and 212257 nonzeros
Model fingerprint: 0xf5d12cfb
Variable types: 42418 continuous, 42585 integer (167 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+08]
  Objective range  [1e+02, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1008 rows and 2004 columns
Presolve time: 0.12s
Presolved: 41832 rows, 82999 columns, 207247 nonzeros
Variable types: 41416 continuous, 41583 integer (4342 binary)
Deterministic concurrent LP optimizer: prim

     0     0 8.5009e+09    0   86 8.5014e+09 8.5009e+09  0.01%     -    6s
     0     0 8.5009e+09    0   85 8.5014e+09 8.5009e+09  0.01%     -    6s
     0     0 8.5009e+09    0   45 8.5014e+09 8.5009e+09  0.01%     -    6s
H    0     0                    8.501423e+09 8.5009e+09  0.01%     -    6s
     0     0 8.5013e+09    0   22 8.5014e+09 8.5013e+09  0.00%     -    6s
     0     0 8.5013e+09    0   19 8.5014e+09 8.5013e+09  0.00%     -    6s
H    0     0                    8.501423e+09 8.5013e+09  0.00%     -    6s
     0     0 8.5014e+09    0    6 8.5014e+09 8.5014e+09  0.00%     -    6s
     0     0 8.5014e+09    0    4 8.5014e+09 8.5014e+09  0.00%     -    6s
     0     0 8.5014e+09    0    4 8.5014e+09 8.5014e+09  0.00%     -    6s
     0     0 8.5014e+09    0    1 8.5014e+09 8.5014e+09  0.00%     -    6s
     0     0 8.5014e+09    0   17 8.5014e+09 8.5014e+09  0.00%     -    6s
     0     0 8.5014e+09    0   17 8.5014e+09 8.5014e+09  0.00%     -    6s
H    0     0             

In [12]:
# Show results.
if m1.status == GRB.OPTIMAL:
    open_plants = [p for p in PSet if x[p].x > 0] # Identify plants that are open.
    flow_edges = [(s, p) for s in SSet for p in PSet if y[s, p].x > TOLERANCE] # Identify flow edges with significant flow.
    total_bioethanol = sum(Yield * y[s, p].x for s, p in flow_edges) # Calculate the total bioethanol produced.
    total_biomass = sum(y[s, p].x for s, p in flow_edges) # Calculate the total biomass used.
    
    # Display the solution details.
    print("\033[1mOptimal Solution:\033[0m")
    print("-"*100)
    print(f"Total cost: ${format(m1.objVal, ',.0f').replace(',', ' ')}")
    print(f"Total bioethanol produced: {format(total_bioethanol, ',.0f').replace(',', ' ')} liters")
    print(f"Total biomass used: {format(total_biomass, ',.0f').replace(',', ' ')} tonne")
    print(f"Total number of open plants: {len(open_plants)}")
    
    # Display the list of open plants.
    print("\n\033[1mOpen Facilities:\033[0m")
    print("{:<15} | {:<10}".format("Type", "Location"))
    print("-"*30)
    for plant in open_plants:
        print("{:<15} | {:<10}".format("Plant", str(plant)))
    
    # Display the biomass flow from suppliers to plants.
    # Split into two parts for better readability in the report. 
    print("\n\033[1mFlow of Biomass from Suppliers to Plants (Part 1):\033[0m")
    print("{:<15} | {:<15} | {:<10}".format("Supplier", "Plant", "Flow (tonne)"))
    print("-"*50)
    half_length = len(flow_edges) // 2
    for idx, (s, p) in enumerate(flow_edges):
        if idx == half_length:
            print("\n\033[1mFlow of Biomass from Suppliers to Plants (Part 2):\033[0m")
            print("{:<15} | {:<15} | {:<10}".format("Supplier", "Plant", "Flow (tonne)"))
            print("-"*50)
        flow = y[s, p].x
        formatted_flow = "{:,.2f}".format(flow).replace(',', ' ')
        print("{:<15} | {:<15} | {:<10}".format(str(s), str(p), formatted_flow))
    
    print("-"*100)
else:
    print("No optimal solution found.")

Optimal Solution:
----------------------------------------------------------------------------------------------------
Total cost: $8 501 421 861
Total bioethanol produced: 500 000 000 liters
Total biomass used: 2 155 172 tonne
Total number of open plants: 11

Open Facilities:
Type            | Location  
------------------------------
Plant           | 541       
Plant           | 9063      
Plant           | 9071      
Plant           | 9102      
Plant           | 9107      
Plant           | 9155      
Plant           | 9178      
Plant           | 9183      
Plant           | 9203      
Plant           | 10058     
Plant           | 10066     

Flow of Biomass from Suppliers to Plants (Part 1):
Supplier        | Plant           | Flow (tonne)
--------------------------------------------------
48001           | 9063            | 13 131.97 
48007           | 10066           | 308.18    
48009           | 9203            | 19 802.14 
48015           | 9178            | 17 336.34 
480

## 2. Task 2

In [13]:
# Create a new model.
m2 = gp.Model("HubSupplyModel")

# Sets:
SSet = suppliers['supplier'].tolist() # Set of suppliers (counties).
PSet = plants['plant'].tolist() # Set of potential plants (biorefineries). 
HSet = hubs['hub'].tolist() # Set of potential hubs.

# Parameters:
Supply = suppliers.set_index('supplier')['supply'].to_dict() # Supply of biomass from supplier s (tonne).
Yield = 232 # Conversion yield of bioethanol (liters per tonne of biomass).
PlantCap = 152063705 # Annual conversion capacity for each plant (liters).
HubCap = 300000 # Annual biomass processing capacity for each hub (tonne).
Demand = 500000000 # Total bioethanol demand (production target) (liters).
InvCost = plants.set_index('plant')['plant_cost'].to_dict() # Cost to maintain a plant at location p.
HubInvCost = hubs.set_index('hub')['hub_cost'].to_dict() # Cost to maintain a hub at location h.
TransCostSH = roads_s_h.set_index(['supplier', 'hub'])['cost_per_unit_s_h'].to_dict() # Transport cost.
TransCostHP = railroads_h_p.set_index(['hub', 'plant'])['cost_per_unit_h_p'].to_dict() # Transport cost.
DistSH = roads_s_h.set_index(['supplier', 'hub'])['dist_s_h'].to_dict() # Distance from s to h (km).
DistHP = railroads_h_p.set_index(['hub', 'plant'])['dist_h_p'].to_dict() # Distance from h to p (km).
LoadCostTruck = 10000 # Combined loading and unloading cost for each truck trip ($).
LoadCostTrain = 60000 # Combined loading and unloading cost for each train trip ($).
TruckCap = 500 # Capacity of truck (tonne).
TrainCap = 20000 # Capacity of train (tonne).

# Decision variables:
x = m2.addVars(PSet, vtype=GRB.BINARY, name="x") # 1 if plant is built at location p, else 0.
z = m2.addVars(HSet, vtype=GRB.BINARY, name="z") # 1 if hub is built at location h, else 0.
ySH = m2.addVars(SSet, HSet, vtype=GRB.CONTINUOUS, name="ySH") # Amount of biomass transported from s to h. 
yHP = m2.addVars(HSet, PSet, vtype=GRB.CONTINUOUS, name="yHP") # Amount of biomass transported from h to p.
tripsSH = m2.addVars(SSet, HSet, vtype=GRB.INTEGER, name="tripsSH") # Number of truck trips needed.
tripsHP = m2.addVars(HSet, PSet, vtype=GRB.INTEGER, name="tripsHP") # Number of train trips needed.

# Objective function:
# Total investment cost for plants.
investment_cost_plant = quicksum(InvCost[p] * x[p] for p in PSet)

# Total investment cost for hubs.
investment_cost_hub = quicksum(HubInvCost[h] * z[h] for h in HSet)

# Total transportation cost from s to h.
transport_cost_SH = quicksum(TransCostSH[s,h] * DistSH[s,h] * ySH[s,h] for s in SSet for h in HSet)

# Total combined loading and unloading cost from s to h.
loading_cost_SH = quicksum(LoadCostTruck * tripsSH[s,h] for s in SSet for h in HSet)

# Total transportation cost from h to p.
transport_cost_HP = quicksum(TransCostHP[h,p] * DistHP[h,p] * yHP[h,p] for h in HSet for p in PSet)

# Total combined loading and unloading cost from h to p.
loading_cost_HP = quicksum(LoadCostTrain * tripsHP[h,p] for h in HSet for p in PSet)

# Set the objective function to minimize the total combined costs.
m2.setObjective(investment_cost_plant + investment_cost_hub 
               + transport_cost_SH + loading_cost_SH 
               + transport_cost_HP + loading_cost_HP
               , GRB.MINIMIZE)

# Constraints:
# Supply constraints: A supplier can only supply the amount of biomass that it has available.
for s in SSet:
    m2.addConstr(quicksum(ySH[s, h] for h in HSet) <= Supply[s], name=f"Supply_{s}")

# Hub constraints: A hub can only receive and send biomass after it is built. 
for h in HSet:
    m2.addConstr(quicksum(ySH[s,h] for s in SSet) <= HubCap * z[h], name=f"HubReceive_{h}")
    m2.addConstr(quicksum(yHP[h,p] for p in PSet) <= HubCap * z[h], name=f"HubSend_{h}")

# Hub balance constraints: Quantity biomass received equals quantity biomass sent.
for h in HSet:
    m2.addConstr(quicksum(ySH[s,h] for s in SSet) == quicksum(yHP[h,p] for p in PSet)
                , name=f"HubBalance_{h}")
    
# Plant constraints: A plant can only process biomass up to its capacity, and only after it is built. 
for p in PSet:
    m2.addConstr(quicksum(Yield * yHP[h,p] for h in HSet) <= PlantCap * x[p], name=f"Plant_{p}")

# Bioethanol production requirement: Ensure that the total bioethanol produced meets the demand.
m2.addConstr(quicksum(Yield * yHP[h, p] for h in HSet for p in PSet) >= Demand, name="Demand")
    
# Truck trips constraints: Ensures the number of truck trips needed.
for s in SSet:
    for h in HSet:
        m2.addConstr(ySH[s,h] <= TruckCap * tripsSH[s,h], name=f"TruckTrips_{s}_{h}")

# Train trips constraints: Ensures the number of train trips needed.
for h in HSet:
    for p in PSet:
        m2.addConstr(yHP[h,p] <= TrainCap * tripsHP[h,p], name=f"TainTrips_{h}_{p}")
        
# Optimize the model.
m2.setParam('MIPGap', 1e-9)
m2.optimize()

Set parameter MIPGap to value 1e-09
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 5 7600 6-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 14414 rows, 27986 columns and 75209 nonzeros
Model fingerprint: 0x5b538e9c
Variable types: 13893 continuous, 14093 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+08]
  Objective range  [8e+00, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 5e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 171 rows and 363 columns
Presolve time: 0.03s
Presolved: 14243 rows, 27623 columns, 60656 nonzeros
Variable types: 13728 continuous, 13895 integer (1025 binary)
Found heuristic solution: objective 5.936334e+09

Root relaxation: objective 4.052721e+09, 427 iterations, 0.01 seconds (0.01 work units)


     0     0 5.0502e+09    0  297 5.1378e+09 5.0502e+09  1.71%     -   10s
     0     0 5.0503e+09    0  302 5.1378e+09 5.0503e+09  1.70%     -   10s
     0     0 5.0574e+09    0  307 5.1378e+09 5.0574e+09  1.56%     -   10s
     0     0 5.0594e+09    0  244 5.1378e+09 5.0594e+09  1.53%     -   10s
     0     0 5.0603e+09    0  323 5.1378e+09 5.0603e+09  1.51%     -   10s
     0     0 5.0604e+09    0  324 5.1378e+09 5.0604e+09  1.51%     -   10s
     0     0 5.0633e+09    0  245 5.1378e+09 5.0633e+09  1.45%     -   11s
H    0     0                    5.137762e+09 5.0633e+09  1.45%     -   13s
     0     0 5.0641e+09    0  250 5.1378e+09 5.0641e+09  1.43%     -   14s
     0     0 5.0644e+09    0  249 5.1378e+09 5.0644e+09  1.43%     -   14s
     0     0 5.0676e+09    0  329 5.1378e+09 5.0676e+09  1.37%     -   14s
     0     0 5.0685e+09    0  257 5.1378e+09 5.0685e+09  1.35%     -   14s
     0     0 5.0688e+09    0  262 5.1378e+09 5.0688e+09  1.34%     -   14s
     0     0 5.0705e+09  

In [14]:
# Show results.
if m2.status == GRB.OPTIMAL:
    open_plants = [p for p in PSet if x[p].x > 0] # Identify plants that are open.
    open_hubs = [h for h in HSet if z[h].x > 0] # Identify hubs that are open.
    flow_edges_hp = [(h, p) for h in HSet for p in PSet if yHP[h, p].x > TOLERANCE] # Identify flow edges with significant flow.
    flow_edges_sh = [(s, h) for s in SSet for h in HSet if ySH[s, h].x > TOLERANCE] # Identify flow edges with significant flow.
    total_bioethanol = sum(Yield * yHP[h,p].x for h,p in flow_edges_hp) # Calculate the total bioethanol produced.
    total_biomass = sum(ySH[s, h].x for s in SSet for h in HSet) # Calculate the total biomass used.
    
    # Display solution details.
    print("\033[1mOptimal Solution:\033[0m")
    print("-"*100)
    print(f"Total cost: ${format(m2.objVal, ',.0f').replace(',', ' ')}")
    print(f"Total bioethanol produced: {format(total_bioethanol, ',.0f').replace(',', ' ')} liters")
    print(f"Total biomass used: {format(total_biomass, ',.0f').replace(',', ' ')} tonne")
    print(f"Total number of open plants: {len(open_plants)}")
    print(f"Total number of open hubs: {len(open_hubs)}")
    
    # Display list of open plants and hubs.
    print("\n\033[1mOpen Facilities:\033[0m")
    print("{:<15} | {:<10}".format("Type", "Location"))
    print("-"*30)
    for plant in open_plants:
        print("{:<15} | {:<10}".format("Plant", str(plant)))
    for hub in open_hubs:
        print("{:<15} | {:<10}".format("Hub", str(hub)))
    
    # Display the biomass flow from suppliers to hubs.
    # Split into two parts for better readability in the report. 
    print("\n\033[1mFlow of Biomass from Suppliers to Hubs (Part 1):\033[0m")
    print("{:<15} | {:<15} | {:<10}".format("Supplier", "Hub", "Flow (tonne)"))
    print("-"*50)
    half_length_sh = len(flow_edges_sh) // 2
    for idx, (s, h) in enumerate(flow_edges_sh):
        if idx == half_length_sh:
            print("\n\033[1mFlow of Biomass from Suppliers to Hubs (Part 2):\033[0m")
            print("{:<15} | {:<15} | {:<10}".format("Supplier", "Hub", "Flow (tonne)"))
            print("-"*50)
        flow_sh = ySH[s, h].x
        flow_sh = "{:,.2f}".format(flow_sh).replace(',', ' ')
        print("{:<15} | {:<15} | {:<10}".format(str(s), str(h), flow_sh))

    # Display biomass flow from hubs to plants. 
    print("\n\033[1mFlow of Biomass from Hubs to Plants:\033[0m")
    print("{:<15} | {:<15} | {:<10}".format("Hub", "Plant", "Flow (tonne)"))
    print("-"*50)
    for h in HSet:
        for p in PSet:
            flow_hp = yHP[h, p].x
            if flow_hp > TOLERANCE:
                flow_hp = "{:,.2f}".format(flow_hp).replace(',', ' ')
                print("{:<15} | {:<15} | {:<10}".format(str(h), str(p), flow_hp))
    print("-"*100)
else:
    print("No optimal solution found.")

Optimal Solution:
----------------------------------------------------------------------------------------------------
Total cost: $5 135 420 807
Total bioethanol produced: 500 000 000 liters
Total biomass used: 2 155 172 tonne
Total number of open plants: 8
Total number of open hubs: 31

Open Facilities:
Type            | Location  
------------------------------
Plant           | 541       
Plant           | 9047      
Plant           | 9060      
Plant           | 9091      
Plant           | 9178      
Plant           | 9183      
Plant           | 9203      
Plant           | 10066     
Hub             | 17201     
Hub             | 17218     
Hub             | 17359     
Hub             | 17372     
Hub             | 17395     
Hub             | 17404     
Hub             | 17447     
Hub             | 17466     
Hub             | 17507     
Hub             | 17592     
Hub             | 17620     
Hub             | 17679     
Hub             | 17717     
Hub             | 17784 

## 3. Task 3

In [15]:
# Create a new model.
m3 = gp.Model("ExtendedHubSupplyModel")

# Sets:
SSet = suppliers['supplier'].tolist() # Set of suppliers (counties).
PSet = plants['plant'].tolist() # Set of potential plants (biorefineries). 
HSet = hubs['hub'].tolist() # Set of potential hubs.
TPSet = ["ThirdParty"] # Set of the single third party supplier.

# Parameters:
Supply = suppliers.set_index('supplier')['supply'].to_dict() # Supply of biomass from supplier s (tonne).
Yield = 232 # Conversion yield of bioethanol (liters per tonne of biomass).
PlantCap = 152063705 # Annual conversion capacity for each plant (liters).
HubCap = 300000 # Annual biomass processing capacity for each hub (tonne).
Demand = 800000000 # Total bioethanol demand (production target) (liters).
InvCost = plants.set_index('plant')['plant_cost'].to_dict() # Cost to maintain a plant at location p.
HubInvCost = hubs.set_index('hub')['hub_cost'].to_dict() # Cost to maintain a hub at location h.
TransCostSH = roads_s_h.set_index(['supplier', 'hub'])['cost_per_unit_s_h'].to_dict() # Transport cost.
TransCostHP = railroads_h_p.set_index(['hub', 'plant'])['cost_per_unit_h_p'].to_dict() # Transport cost.
DistSH = roads_s_h.set_index(['supplier', 'hub'])['dist_s_h'].to_dict() # Distance from s to h (km).
DistHP = railroads_h_p.set_index(['hub', 'plant'])['dist_h_p'].to_dict() # Distance from h to p (km).
LoadCostTruck = 10000 # Combined loading and unloading cost for each truck trip ($).
LoadCostTrain = 60000 # Combined loading and unloading cost for each train trip ($).
TruckCap = 500 # Capacity of truck (tonne).
TrainCap = 20000 # Capacity of train (tonne).
ThirdPartyCost = 2000 # Cost of 1 tonne biomass from the third party supplier ($).

# Decision variables:
x = m3.addVars(PSet, vtype=GRB.BINARY, name="x") # 1 if plant is built at location p, else 0.
z = m3.addVars(HSet, vtype=GRB.BINARY, name="z") # 1 if hub is built at location h, else 0.
ySH = m3.addVars(SSet, HSet, vtype=GRB.CONTINUOUS, name="ySH") # Amount of biomass transported from s to h. 
yHP = m3.addVars(HSet, PSet, vtype=GRB.CONTINUOUS, name="yHP") # Amount of biomass transported from h to p.
yTP = m3.addVars(TPSet, HSet, vtype=GRB.CONTINUOUS, name="yTP") # Amount of biomass from third party supplier.
tripsSH = m3.addVars(SSet, HSet, vtype=GRB.INTEGER, name="tripsSH") # Number of truck trips needed.
tripsHP = m3.addVars(HSet, PSet, vtype=GRB.INTEGER, name="tripsHP") # Number of train trips needed.

# Objective function:
# Total investment cost for plants.
investment_cost_plant = quicksum(InvCost[p] * x[p] for p in PSet)

# Total investment cost for hubs.
investment_cost_hub = quicksum(HubInvCost[h] * z[h] for h in HSet)

# Total transportation cost from s to h.
transport_cost_SH = quicksum(TransCostSH[s,h] * DistSH[s,h] * ySH[s,h] for s in SSet for h in HSet)

# Total combined loading and unloading cost from s to h.
loading_cost_SH = quicksum(LoadCostTruck * tripsSH[s,h] for s in SSet for h in HSet)

# Total transportation cost from h to p.
transport_cost_HP = quicksum(TransCostHP[h,p] * DistHP[h,p] * yHP[h,p] for h in HSet for p in PSet)

# Total combined loading and unloading cost from h to p.
loading_cost_HP = quicksum(LoadCostTrain * tripsHP[h,p] for h in HSet for p in PSet)

# Total cost for biomass from the third party supplier to hub h.
third_party_cost = quicksum(ThirdPartyCost * yTP[tp,h] for tp in TPSet for h in HSet)

# Set the objective function to minimize the total combined costs.
m3.setObjective(investment_cost_plant + investment_cost_hub 
               + transport_cost_SH + loading_cost_SH 
               + transport_cost_HP + loading_cost_HP
               + third_party_cost
               , GRB.MINIMIZE)

# Constraints:
# Supply constraints: A supplier can only supply the amount of biomass that it has available.
for s in SSet:
    m3.addConstr(quicksum(ySH[s, h] for h in HSet) <= Supply[s], name=f"Supply_{s}")

# Hub constraints: A hub can only receive and send biomass after it is built. 
for h in HSet:
    m3.addConstr(quicksum(ySH[s,h] for s in SSet) + yTP["ThirdParty", h] <= HubCap * z[h], name=f"HubReceive_{h}")
    m3.addConstr(quicksum(yHP[h,p] for p in PSet) <= HubCap * z[h], name=f"HubSend_{h}")

# Hub balance constraints: Quantity biomass received equals quantity biomass sent.
for h in HSet:
    m3.addConstr(quicksum(ySH[s,h] for s in SSet) + yTP["ThirdParty",h] == quicksum(yHP[h,p] for p in PSet)
                , name=f"HubBalance_{h}")
    
# Plant constraints: A plant can only process biomass up to its capacity, and only after it is built. 
for p in PSet:
    m3.addConstr(quicksum(Yield * yHP[h,p] for h in HSet) <= PlantCap * x[p], name=f"Plant_{p}")

# Bioethanol production requirement: Ensure that the total bioethanol produced meets the demand.
m3.addConstr(quicksum(Yield * yHP[h, p] for h in HSet for p in PSet) >= Demand, name="Demand")

# Truck trips constraints: Ensures the number of truck trips needed.
for s in SSet:
    for h in HSet:
        m3.addConstr(ySH[s,h] <= TruckCap * tripsSH[s,h], name=f"TruckTrips_{s}_{h}")

# Train trips constraints: Ensures the number of train trips needed.
for h in HSet:
    for p in PSet:
        m3.addConstr(yHP[h,p] <= TrainCap * tripsHP[h,p], name=f"TrainTrips{h}_{p}")
        
# Optimize the model.
m3.setParam('MIPGap', 1e-9)
m3.optimize()

Set parameter MIPGap to value 1e-09
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 5 7600 6-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 14414 rows, 28019 columns and 75275 nonzeros
Model fingerprint: 0xe9766651
Variable types: 13926 continuous, 14093 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+08]
  Objective range  [8e+00, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 8e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 8354 rows and 16318 columns
Presolve time: 0.03s
Presolved: 6060 rows, 11701 columns, 34433 nonzeros
Variable types: 5767 continuous, 5934 integer (205 binary)
Found heuristic solution: objective 7.978678e+09

Root relaxation: objective 6.517144e+09, 344 iterations, 0.01 seconds (0.00 work units)



In [16]:
# Show results.
if m3.status == GRB.OPTIMAL:
    open_plants = [p for p in PSet if x[p].x > 0] # Identify plants that are open.
    open_hubs = [h for h in HSet if z[h].x > 0] # Identify hubs that are open.
    flow_edges_hp = [(h, p) for h in HSet for p in PSet if yHP[h, p].x > TOLERANCE] # Identify flow edges with significant flow.
    flow_edges_sh = [(s, h) for s in SSet for h in HSet if ySH[s, h].x > TOLERANCE] # Identify flow edges with significant flow.
    total_bioethanol = sum(Yield * yHP[h,p].x for h,p in flow_edges_hp) # Calculate the total bioethanol produced.
    total_third_party_biomass = sum(yTP["ThirdParty", h].x for h in HSet) # Calculate the total biomass sourced from third party.
    total_biomass_excl_third_party = sum(ySH[s, h].x for s in SSet for h in HSet) # Calculate the total biomass exluding from third party.
    total_biomass_incl_third_party = total_biomass_excl_third_party + total_third_party_biomass # Calculate total biomass including third party.
    
    # Display solution details.
    print("\033[1mOptimal Solution:\033[0m")
    print("-"*100)
    print(f"Total Cost: ${format(m3.objVal, ',.0f').replace(',', ' ')}")
    print(f"Total bioethanol produced: {format(total_bioethanol, ',.0f').replace(',', ' ')} liters")
    print(f"Total biomass used (including third party): {format(total_biomass_incl_third_party, ',.0f').replace(',', ' ')} tonne")
    print(f"Total biomass sourced from third party supplier: {format(total_third_party_biomass, ',.0f').replace(',', ' ')} tonne")
    print(f"Total biomass used (excluding third party): {format(total_biomass_excl_third_party, ',.0f').replace(',', ' ')} tonne")
    print(f"Total number of open plants: {len(open_plants)}")
    print(f"Total number of open hubs: {len(open_hubs)}")

    # Display list of open plants and hubs.
    print("\n\033[1mOpen Facilities:\033[0m")
    print("{:<15} | {:<10}".format("Type", "Location"))
    print("-"*30)
    for plant in open_plants:
        print("{:<15} | {:<10}".format("Plant", str(plant)))
    for hub in open_hubs:
        print("{:<15} | {:<10}".format("Hub", str(hub)))
    
    # Display the biomass flow from third party supplier to hubs.
    print("\n\033[1mFlow of Biomass from Third Party Supplier to Hubs:\033[0m")
    print("{:<15} | {:<15} | {:<10}".format("Third Party", "Hub", "Flow (tonne)"))
    print("-"*50)
    for h in HSet:
        third_party_flow = yTP["ThirdParty", h].x
        if third_party_flow > TOLERANCE: 
            formatted_flow = "{:,.2f}".format(third_party_flow).replace(',', ' ')
            print("{:<15} | {:<15} | {:<10}".format("ThirdParty", str(h), formatted_flow))
    
    # Display the biomass flow from suppliers to hubs.
    # Split into two parts for better readability in the report. 
    print("\n\033[1mFlow of Biomass from Suppliers to Hubs (Part 1):\033[0m")
    print("{:<15} | {:<15} | {:<10}".format("Supplier", "Hub", "Flow (tonne)"))
    print("-"*50)
    half_length_sh = len(flow_edges_sh) // 2
    for idx, (s, h) in enumerate(flow_edges_sh):
        if idx == half_length_sh:
            print("\n\033[1mFlow of Biomass from Suppliers to Hubs (Part 2):\033[0m")
            print("{:<15} | {:<15} | {:<10}".format("Supplier", "Hub", "Flow (tonne)"))
            print("-"*50)
        flow_sh = ySH[s, h].x
        flow_sh = "{:,.2f}".format(flow_sh).replace(',', ' ')
        print("{:<15} | {:<15} | {:<10}".format(str(s), str(h), flow_sh))

    # Display biomass flow from hubs to plants. 
    print("\n\033[1mFlow of Biomass from Hubs to Plants:\033[0m")
    print("{:<15} | {:<15} | {:<10}".format("Hub", "Plant", "Flow (tonne)"))
    print("-"*50)
    for h in HSet:
        for p in PSet:
            flow_hp = yHP[h, p].x
            if flow_hp > TOLERANCE:
                flow_hp = "{:,.2f}".format(flow_hp).replace(',', ' ')
                print("{:<15} | {:<15} | {:<10}".format(str(h), str(p), flow_hp))
    print("-"*100)
else:
    print("No optimal solution found.")

Optimal Solution:
----------------------------------------------------------------------------------------------------
Total Cost: $7 144 532 489
Total bioethanol produced: 800 000 000 liters
Total biomass used (including third party): 3 448 276 tonne
Total biomass sourced from third party supplier: 2 052 787 tonne
Total biomass used (excluding third party): 1 395 489 tonne
Total number of open plants: 7
Total number of open hubs: 23

Open Facilities:
Type            | Location  
------------------------------
Plant           | 9044      
Plant           | 9047      
Plant           | 9060      
Plant           | 9178      
Plant           | 9183      
Plant           | 9203      
Plant           | 10066     
Hub             | 17201     
Hub             | 17218     
Hub             | 17359     
Hub             | 17395     
Hub             | 17404     
Hub             | 17447     
Hub             | 17507     
Hub             | 17592     
Hub             | 17679     
Hub             | 17